In [1]:
%cd ./../.. 

/home/iir/work/ben/NCKU/IIR


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
# Define the BERT tokenizer and model
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
name_bert = "alvaroalon2/biobert_diseases_ner"
tokenizer = BertTokenizer.from_pretrained(name_bert)

/root/anaconda3/envs/augllms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/augllms/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
path_data_train = "IIR_SemEval-2013-task-9/IIR_Preprocessing/save_results/train_aug_llamba.csv"
path_data_test = "IIR_SemEval-2013-task-9/IIR_Preprocessing/save_results/test_dataset.csv"
path_data_devel = "IIR_SemEval-2013-task-9/IIR_Preprocessing/save_results/Devel_dataset.csv"
train_df = pd.read_csv(path_data_train, sep=',', encoding='utf-8')
test_df = pd.read_csv(path_data_test, sep=',', encoding='utf-8')
devel_df = pd.read_csv(path_data_devel, sep=',', encoding='utf-8')
le = LabelEncoder()
class_mapping = {
    'false': 0,
    'effect': 1,
    'mechanism': 2,
    'advise': 3,
    'int': 4
}
# Fit and transform the 'pair type' column using the custom mapping
train_df['pair type'] = le.fit_transform(train_df['pair type'].map(class_mapping))
test_df['pair type'] = le.fit_transform(test_df['pair type'].map(class_mapping))
devel_df['pair type'] = le.fit_transform(devel_df['pair type'].map(class_mapping))
# Use train_test_split with stratify
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['pair type'], random_state=42)

In [4]:
train_df.shape, test_df.shape, devel_df.shape, val_df.shape

((58548, 5), (5716, 4), (2703, 4), (6506, 5))

In [5]:
# Tokenize and encode the sentences
class MyDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data.iloc[index]['Full Sentence']
        e1 = self.data.iloc[index]['entity e1']
        e2 = self.data.iloc[index]['entity e2']
        pair_type = self.data.iloc[index]['pair type']
#         input_text = f"{e1} [SEP] {e2}"
        input_text = f"{sentence} [SEP] {e1} [SEP] {e2}"

        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': input_text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(pair_type, dtype=torch.long)
        }


In [6]:
# create an instance of the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(name_bert)
# set the batch size
batch_size = 400
num_epochs = 150  # Adjust as needed

# set the maximum sequence length
max_len = 50

# create data loaders for the training and validation sets
train_dataset = MyDataset(train_df, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

devel_dataset = MyDataset(devel_df, tokenizer, max_len)
devel_loader = DataLoader(devel_dataset, batch_size=batch_size)

test_dataset = MyDataset(test_df, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

val_dataset = MyDataset(val_df, tokenizer, max_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

/root/anaconda3/envs/augllms/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
class BertSentimentClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BertSentimentClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 256)
        self.dropout2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, num_classes)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout1(pooled_output)
        output = self.fc1(pooled_output)
        output = self.dropout2(output)
        logits = self.fc2(output)

        return logits

In [8]:

num_classes = train_df["pair type"].nunique()
model = BertSentimentClassifier(name_bert, num_classes)
criterion = nn.CrossEntropyLoss()

/root/anaconda3/envs/augllms/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at alvaroalon2/biobert_diseases_ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch.nn.functional as F

def focal_loss(outputs, labels, gamma=2.0, alpha=1.0):
    ce_loss = F.cross_entropy(outputs, labels, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = (alpha * (1 - pt) ** gamma * ce_loss).mean()
    return focal_loss

def train(model, data_loader, optimizer, criterion, device):
    # set the model to train mode
    model.train()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss, total_accuracy = 0, 0
    total_precision, total_recall, total_f1 = 0, 0, 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # zero the gradient
        optimizer.zero_grad()

        # get the model's predictions
        outputs = model(input_ids, attention_mask)

        # get the focal loss
        loss = focal_loss(outputs, labels)
        total_loss += loss.item()

        # accuracy calculation
        logits = outputs.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        total_accuracy += flat_accuracy(logits, label_ids)

        # calculate precision, recall, and f1 score
        total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

        # perform backpropagation and optimization
        loss.backward()
        optimizer.step()

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # avg_accuracy = total_accuracy / len(data_loader)
    avg_precision = total_precision / len(data_loader)
    avg_recall = total_recall / len(data_loader)
    avg_f1 = total_f1 / len(data_loader)

    return avg_loss, avg_precision, avg_recall, avg_f1


In [10]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [11]:
def evaluate(model, data_loader, criterion, device):
    # set the model to eval mode
    model.eval()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # disable gradient computation
        with torch.no_grad():
            # get the model's predictions
            outputs = model(input_ids, attention_mask)

            # get the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # calculate the number of correct predictions
            logits = outputs.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            total_accuracy += flat_accuracy(logits, label_ids)

            # calculate precision, recall, and f1 score
            total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # accuracy = total_accuracy / len(data_loader)
    precision = total_precision / len(data_loader)
    recall = total_recall / len(data_loader)
    f1 = total_f1 / len(data_loader)

    return avg_loss, precision, recall, f1

In [12]:
# Function to calculate F1 score, Precision, Recall
def calculate_metrics(predictions, labels):
    predictions = torch.argmax(predictions, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return f1, precision, recall

In [13]:
import warnings

# ...

# Ignore warnings
warnings.filterwarnings("ignore")

# Create a folder for logs if it doesn't exist
logs_folder = "logs"
os.makedirs(logs_folder, exist_ok=True)

# Generate a filename based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"{logs_folder}/training_log_BERT1_{current_time}.txt"



# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set the learning rate
lr = 2e-5

# set the number of epochs


# set the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# move the model to the device
model.to(device)

# lists to store metrics
train_losses = []
train_precisions = []
train_recalls = []
train_f1_scores = []

devel_losses = []
devel_precisions = []
devel_recalls = []
devel_f1_scores = []

test_losses = []
test_precisions = []
test_recalls = []
test_f1_scores = []

val_losses = []
val_precisions = []
val_recalls = []
val_f1_scores = []

best_test_loss = float('inf')
best_epoch = 0

# iterate over the epochs
# Set up logging
with open(log_filename, 'a') as log_file:
  for epoch in range(num_epochs):
      start_time = datetime.now()
      print(f'Epoch {epoch + 1}/{num_epochs}')

      # train the model
      train_loss, train_precision, train_recall, train_f1 = train(model, train_loader, optimizer, criterion, device)

      # evaluate the model on the validation set
      val_loss, val_precision, val_recall, val_f1 = evaluate(model, val_loader, criterion, device)

      # evaluate the model on the validation set
      devel_loss, devel_precision, devel_recall, devel_f1 = evaluate(model, devel_loader, criterion, device)

          # evaluate the model on the validation set
      test_loss, test_precision, test_recall, test_f1 = evaluate(model, test_loader, criterion, device)

      # print the detailed metrics
      print(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}')
      print(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
      print(f'Devel -- Loss: {devel_loss:.4f} | Precision: {devel_precision:.4f} | Recall: {devel_recall:.4f} | F1: {devel_f1:.4f}')
      print(f'Test  -- Loss: {test_loss:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f} | F1: {test_f1:.4f}')

      # log the metrics
      log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
      log_file.write(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}\n')
      log_file.write(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
      log_file.write(f'Devel -- Loss: {devel_loss:.4f} | Precision: {devel_precision:.4f} | Recall: {devel_recall:.4f} | F1: {devel_f1:.4f}\n')
      log_file.write(f'Test  -- Loss: {test_loss:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f} | F1: {test_f1:.4f}\n')
      log_file.flush()

      # Save the best model based on validation loss
      if test_loss < best_test_loss:
          best_test_loss = test_loss
          best_epoch = epoch + 1
          torch.save(model.state_dict(), f'best_model_BERT1.pt')

      # Save the metrics
      train_losses.append(train_loss)
      train_precisions.append(train_precision)
      train_recalls.append(train_recall)
      train_f1_scores.append(train_f1)

      val_losses.append(val_loss)
      val_precisions.append(val_precision)
      val_recalls.append(val_recall)
      val_f1_scores.append(val_f1)

      devel_losses.append(devel_loss)
      devel_precisions.append(devel_precision)
      devel_recalls.append(devel_recall)
      devel_f1_scores.append(devel_f1)

      test_losses.append(test_loss)
      test_precisions.append(test_precision)
      test_recalls.append(test_recall)
      test_f1_scores.append(test_f1)

      # Calculate and print time remaining
      elapsed_time = datetime.now() - start_time
      remaining_time = (num_epochs - epoch - 1) * elapsed_time
      print(f'Time Elapsed: {elapsed_time}, Time Remaining: {remaining_time} \n')
      torch.save(model.state_dict(), f'final_model_BERT1.pt')

# Print the best epoch
print(f"Best Model: Epoch {best_epoch}, Best Validation Loss: {best_test_loss:.4f}")

print(f"Path log file: ", log_filename)



Epoch 1/150


100%|██████████| 15/15 [00:05<00:00,  2.76it/s]


Train -- Loss: 0.3654 | Precision: 0.7037 | Recall: 0.7501 | F1: 0.7094
Val   -- Loss: 0.4354 | Precision: 0.8707 | Recall: 0.8625 | F1: 0.8566
Devel -- Loss: 0.5525 | Precision: 0.7652 | Recall: 0.7660 | F1: 0.7487
Test  -- Loss: 0.5960 | Precision: 0.7800 | Recall: 0.7839 | F1: 0.7676
Time Elapsed: 0:01:25.438224, Time Remaining: 3:32:10.295376 

Epoch 2/150


100%|██████████| 15/15 [00:05<00:00,  2.66it/s]


Train -- Loss: 0.1170 | Precision: 0.8709 | Recall: 0.8686 | F1: 0.8666
Val   -- Loss: 0.3500 | Precision: 0.8935 | Recall: 0.8881 | F1: 0.8848
Devel -- Loss: 0.5591 | Precision: 0.7828 | Recall: 0.7706 | F1: 0.7581
Test  -- Loss: 0.6218 | Precision: 0.7782 | Recall: 0.7578 | F1: 0.7597
Time Elapsed: 0:01:21.262015, Time Remaining: 3:20:26.778220 

Epoch 3/150


...

100%|██████████| 15/15 [00:05<00:00,  2.79it/s]


Train -- Loss: 0.0532 | Precision: 0.9224 | Recall: 0.9194 | F1: 0.9182
Val   -- Loss: 0.2503 | Precision: 0.9201 | Recall: 0.9143 | F1: 0.9121
Devel -- Loss: 0.5633 | Precision: 0.8328 | Recall: 0.7981 | F1: 0.7946
Test  -- Loss: 0.6562 | Precision: 0.8119 | Recall: 0.7709 | F1: 0.7796
Time Elapsed: 0:01:21.555708, Time Remaining: 3:04:51.576288 

Epoch 15/150


100%|██████████| 15/15 [00:05<00:00,  2.68it/s]


Train -- Loss: 0.0526 | Precision: 0.9238 | Recall: 0.9205 | F1: 0.9194
Val   -- Loss: 0.2488 | Precision: 0.9173 | Recall: 0.9126 | F1: 0.9109
Devel -- Loss: 0.6084 | Precision: 0.8228 | Recall: 0.7847 | F1: 0.7865
Test  -- Loss: 0.7880 | Precision: 0.8088 | Recall: 0.7424 | F1: 0.7571
Time Elapsed: 0:01:25.070428, Time Remaining: 3:11:24.507780 

Epoch 16/150


  6%|▌         | 9/147 [00:04<01:16,  1.81it/s]